In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

<ipython-input-2-63192117ceec>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/GCN_minmax_baseline_version4"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}

## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # RNN layer 1
        x = layers.SimpleRNN(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True
        )(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # RNN layer 2
        x = layers.SimpleRNN(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=False
        )(x)
        x = layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'rnn_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 03m 02s]
val_accuracy: 0.992068350315094

Best val_accuracy So Far: 0.9926784634590149
Total elapsed time: 00h 33m 43s
Best Parameters for Week 1:
units_1: 64
dropout_1: 0.1
units_2: 128
dropout_2: 0.2
learning_rate: 0.001426144356481806

Best Parameters for Week 2:
units_1: 192
dropout_1: 0.1
units_2: 32
dropout_2: 0.4
learning_rate: 0.0006797523206751302

Best Parameters for Week 3:
units_1: 64
dropout_1: 0.4
units_2: 160
dropout_2: 0.4
learning_rate: 0.0003156343327585864

Best Parameters for Week 4:
units_1: 96
dropout_1: 0.1
units_2: 192
dropout_2: 0.4
learning_rate: 0.00011631756169915276


## Danh sách tham số tốt nhất của từng tuần

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình BiLSTM
def build_RNN_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # RNN layer 1
    x = layers.SimpleRNN(
        units=params.get('units_1'),
        return_sequences=True
    )(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # RNN layer 2
    x = layers.SimpleRNN(
        units=params.get('units_2', 32),
        return_sequences=False
    )(x)
    x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
        input_shape = (X_train.shape[1], X_train.shape[2])  # Lấy kích thước từ dữ liệu thực tế
        # Xây dựng mô hình với tham số tốt nhất
        model = build_RNN_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 64, 'dropout_1': 0.1, 'units_2': 128, 'dropout_2': 0.2, 'learning_rate': 0.001426144356481806}
Fold 1: Using file /kaggle/input/GCN_minmax_baseline_version4/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - accuracy: 0.5627 - loss: 0.1627 - val_accuracy: 0.6805 - val_loss: 0.1236
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6415 - loss: 0.1374 - val_accuracy: 0.6813 - val_loss: 0.1264
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6591 - loss: 0.1265 - val_accuracy: 0.7164 - val_loss: 0.1068
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6887 - loss: 0.1188 - val_accuracy: 0.7484 - val_loss: 0.0988
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7039 - loss: 0.1127 - val_accuracy: 0.7503 - val_loss: 0.1005
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 28ms/step - accuracy: 0.5995 - loss: 0.1582 - val_accuracy: 0.6660 - val_loss: 0.1233
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6407 - loss: 0.1318 - val_accuracy: 0.6794 - val_loss: 0.1174
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6645 - loss: 0.1254 - val_accuracy: 0.7083 - val_loss: 0.1092
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6807 - loss: 0.1150 - val_accuracy: 0.7106 - val_loss: 0.1089
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6932 - loss: 0.1143 - val_accuracy: 0.7495 - val_loss: 0.0970
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7184 - loss: 0.1071 - val_accuracy: 0.7488 - val_loss: 0.0989
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7247 - loss: 0.1041 - val_accuracy: 0.7644 - val_loss: 0.0951
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7280 - loss: 0.1024 - val_accuracy: 0.7400 - va

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.5994 - loss: 0.1658 - val_accuracy: 0.6659 - val_loss: 0.1257
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6440 - loss: 0.1309 - val_accuracy: 0.5950 - val_loss: 0.1241
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6642 - loss: 0.1213 - val_accuracy: 0.6629 - val_loss: 0.1137
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6877 - loss: 0.1166 - val_accuracy: 0.7044 - val_loss: 0.1084
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7062 - loss: 0.1107 - val_accuracy: 0.7021 - val_loss: 0.1098
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7240 - loss: 0.1061 - val_accuracy: 0.6983 - val_loss: 0.1062
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7347 - loss: 0.1049 - val_accuracy: 0.7113 - val_loss: 0.1224
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7262 - loss: 0.1018 - val_accuracy: 0.7433 - va

...

Fold 4: Using file /kaggle/input/GCN_minmax_baseline_version4/clean_week3/train/5-folds/data_part_4.csv as test set
Epoch 1/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - accuracy: 0.5054 - loss: 0.2073 - val_accuracy: 0.6648 - val_loss: 0.1310
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5993 - loss: 0.1623 - val_accuracy: 0.6907 - val_loss: 0.1230
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5949 - loss: 0.1498 - val_accuracy: 0.6998 - val_loss: 0.1217
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6249 - loss: 0.1403 - val_accuracy: 0.6979 - val_loss: 0.1203
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6202 - loss: 0.1415 - val_accuracy: 0.7262 - val_loss: 0.1188
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6481 - loss: 0.1327 - val_accuracy: 0.7410 - val_loss: 0.1059
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6742 - loss: 0.1277 - val_accuracy: 0.7784 - val

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


328/328 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.4810 - loss: 0.2270 - val_accuracy: 0.6720 - val_loss: 0.1312
Epoch 2/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5836 - loss: 0.1653 - val_accuracy: 0.6709 - val_loss: 0.1268
Epoch 3/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5995 - loss: 0.1518 - val_accuracy: 0.6895 - val_loss: 0.1286
Epoch 4/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6161 - loss: 0.1422 - val_accuracy: 0.6812 - val_loss: 0.1208
Epoch 5/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6208 - loss: 0.1410 - val_accuracy: 0.6937 - val_loss: 0.1132
Epoch 6/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6618 - loss: 0.1294 - val_accuracy: 0.7418 - val_loss: 0.1030
Epoch 7/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6836 - loss: 0.1219 - val_accuracy: 0.7849 - val_loss: 0.0947
Epoch 8/20
328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7128 - loss: 0.1130 - val_accuracy: 0.7750 - va

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



=== Average Accuracy ===
0.8311

=== Average Macro Metrics ===
Macro Precision: 0.5756
Macro Recall: 0.5354
Macro F1-Score: 0.5461
Macro AUC-ROC: 0.9196

=== Average Weighted Metrics ===
Weighted Precision: 0.8015
Weighted Recall: 0.8311
Weighted F1-Score: 0.8111
Weighted AUC-ROC: 0.9565

=== Average Metrics per Label ===
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.708563        0.802000          0.745455     0.949276
1      1           0.000000        0.000000          0.000000     0.866529
2      2           0.618609        0.431870          0.506327     0.887222
3      3           0.634390        0.477844          0.539176     0.919794
4      4           0.916199        0.965075          0.939512     0.975007

=== Average Confusion Matrix ===
       0    1     2     3       4
0  481.2  0.0  27.2  27.4    64.2
1   40.2  0.0   3.0   4.2    40.2
2   61.2  0.0  71.0  10.6    21.6
3   63.6  0.0   5.4  79.8    18.2
4   41.2  0.0   9.6  

## Kết quả cross validation trên 5-folds

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.8868979096412659
Average Train Time: 24.7376 seconds
Average Test Time: 0.7731 seconds
Average AUC Macro: 0.9588051345557543
Average AUC Weighted: 0.9824601392733037

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.594130        0.671333          0.626373     0.875844
1      1           0.000000        0.000000          0.000000     0.820541
2      2           0.612532        0.446467          0.513926     0.879130
3      3           0.584855        0.428743          0.484626     0.900553
4      4           0.868587        0.914298          0.889997     0.936989

Average Confusion Matrix:
       0    1     2     3       4
0  402.8  0.0  26.4  26.8   144.0
1   55.4  0.0   3.0   6.0    23.2
2   55.0  0.0  73.4   8.4    27.6
3   62.8  0.0   3.6  71.6    29.0
4  111.8  0.0  14.2  11.4  1466.0

=== Results for week2 ===
Average Accurancy: 0.

## Kiểm tra trên tập test

In [11]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])  # Lấy kích thước từ dữ liệu thực tế
    model = build_RNN_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=20, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [12]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.4195 - loss: 0.2248 - val_accuracy: 0.4423 - val_loss: 0.3102
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5329 - loss: 0.1797 - val_accuracy: 0.4021 - val_loss: 0.2978
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5585 - loss: 0.1669 - val_accuracy: 0.4702 - val_loss: 0.2220
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5647 - loss: 0.1625 - val_accuracy: 0.4557 - val_loss: 0.2416
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5762 - loss: 0.1579 - val_accuracy: 0.3612 - val_loss: 0.3074
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5796 - loss: 0.1543 - val_accuracy: 0.5298 - val_loss: 0.2529
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5911 - loss: 0.1495 - val_accuracy: 0.6565 - val_loss: 0.1534
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [13]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.4094 - loss: 0.2333 - val_accuracy: 0.4298 - val_loss: 0.3289
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5471 - loss: 0.1766 - val_accuracy: 0.4363 - val_loss: 0.2429
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6243 - loss: 0.1432 - val_accuracy: 0.6558 - val_loss: 0.1653
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7097 - loss: 0.1071 - val_accuracy: 0.6383 - val_loss: 0.1463
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7536 - loss: 0.0886 - val_accuracy: 0.5922 - val_loss: 0.1265
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7815 - loss: 0.0778 - val_accuracy: 0.5717 - val_loss: 0.1301
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7989 - loss: 0.0710 - val_accuracy: 0.2644 - val_loss: 0.2070
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [14]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.3119 - loss: 0.2826 - val_accuracy: 0.4136 - val_loss: 0.3356
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4093 - loss: 0.2194 - val_accuracy: 0.4138 - val_loss: 0.3404
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4695 - loss: 0.1920 - val_accuracy: 0.4378 - val_loss: 0.3051
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5091 - loss: 0.1751 - val_accuracy: 0.4807 - val_loss: 0.2205
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5580 - loss: 0.1616 - val_accuracy: 0.5258 - val_loss: 0.1909
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6139 - loss: 0.1401 - val_accuracy: 0.5316 - val_loss: 0.1838
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6751 - loss: 0.1139 - val_accuracy: 0.5605 - val_loss: 0.1225
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [15]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.3515 - loss: 0.2622 - val_accuracy: 0.4300 - val_loss: 0.2913
Epoch 2/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4877 - loss: 0.2023 - val_accuracy: 0.4343 - val_loss: 0.2686
Epoch 3/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5556 - loss: 0.1693 - val_accuracy: 0.4430 - val_loss: 0.2213
Epoch 4/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6481 - loss: 0.1326 - val_accuracy: 0.5326 - val_loss: 0.1932
Epoch 5/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7112 - loss: 0.1081 - val_accuracy: 0.6363 - val_loss: 0.1527
Epoch 6/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7626 - loss: 0.0867 - val_accuracy: 0.6211 - val_loss: 0.1561
Epoch 7/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8018 - loss: 0.0705 - val_accuracy: 0.7428 - val_loss: 0.1021
Epoch 8/20
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accura

In [16]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 48.12 seconds
  Test Time: 1.05 seconds
  Accurancy: 0.7579268292682927
  Precision: [0.65186916 0.32258065 0.37654321 0.58       0.95099183]
  Recall: [0.744      0.55555556 0.59223301 0.55238095 0.81256231]
  F1-Score: [0.69489415 0.40816327 0.46037736 0.56585366 0.87634409]
  Macro Precision: 0.5763969691776667
  Macro Recall: 0.6513463661412127
  Macro F1-Score: 0.6011265030607442
  Confusion Matrix:
[[279  22  39  14  21]
 [  8  30  10   1   5]
 [ 26   4  61   6   6]
 [  8   9  20  58  10]
 [107  28  32  21 815]]

=== AUC-ROC ===
AUC Macro: 0.9646
AUC Weighted: 0.9850
AUC per Label: [0.98258737 0.95159916 0.9355911  0.95788139 0.99558605]
Week 2:
  Train Time: 47.01 seconds
  Test Time: 1.03 seconds
  Accurancy: 0.9231707317073171
  Precision: [0.91168091 0.89830508 0.775      0.90425532 0.94783465]
  Recall: [0.85333333 0.98148148 0.90291262 0.80952381 0.96011964]
  F1-Score: [0.8815427  0.9380531  0.83408072 0.85427136